In [13]:
import os
from os import listdir
from PIL import Image
from numpy import asarray
from numpy import expand_dims
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
from keras.models import load_model
import numpy as np

import pickle

In [14]:
detector = MTCNN()
model = load_model('facenet_keras.h5')

In [15]:
def extract_face(filename):
    image = Image.open(filename)                    
    image = image.convert('RGB')
    pixels = asarray(image)   
    
    wajah = detector.detect_faces(pixels)        
    
    if len(wajah)>0:
        x1, y1, width, height = wajah[0]['box']         
    else:
        x1=1
        y1=1
        width=10
        height=10
        
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]                         # extract the face
    
    image = Image.fromarray(face)                       # resize pixels to the model size
    image = image.resize((160,160))
    face_array = asarray(image)
    return face_array

In [18]:
folder='train/'
database = {}

for filename in listdir(folder):

    path = folder + filename
    face = extract_face(path)
    
    face_pixels = face.astype('float32')
    mean, std = face.mean(), face.std()
    face = (face - mean) / std
    
    face = expand_dims(face, axis=0)
    signature = model.predict(face)
    
    database[os.path.splitext(filename)[0]]=signature


In [19]:
myfile = open("data.pkl", "wb")
pickle.dump(database, myfile)
myfile.close()